In [ ]:

! pip install mistralai evaluate openai google-generativeai

# Summary:
* openai api is unusable due to low limits and quotas
* gemini api returns garbage so i had to remove ```json and ``` from the responses and change the prompt a little bit to make it somewhat work
* mistral api works fine but sometimes does not fit the best product to review
* running this notebook is ill advised as it took near an hour to get all the responses

In [ ]:
from google.colab import userdata

#userdata.get('MISTRAL_API_KEY')
mistral_key = userdata.get('mistral')
open_key = userdata.get('gpt')
gemini_key = userdata.get('gemini')

In [ ]:
from mistralai import Mistral
from openai import OpenAI
import google.generativeai as genai
#from getpass import getpass

#api_key= getpass("Type your API Key")

mistral_client = Mistral(api_key = mistral_key)
openai_client = OpenAI(api_key = open_key)

genai.configure(api_key = gemini_key)
gemini_client = genai.GenerativeModel("gemini-1.5-flash")


In [ ]:
prompts = {
    "review1": {
        "review_content": "Gets my clothes fresh and clean every time. No lingering odor with Tide.",
        "golden_answer": {
            "product category": "Powder Detergents for Laundry",
            "brand": "Tide",
            "other keywords": ['fresh', 'clean', 'no lingering odor']
        },
    },
    "review2": {
        "review_content": "Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.",
        "golden_answer": {
            "product category": "Thermal Mugs",
            "brand": "Contigo",
            "other keywords": ["hot for hours"]
        },
    },
}

In [ ]:
reviews = [
    # Thermal Mugs
    "Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.",
    "The lid isn’t leak-proof, but it keeps drinks warm for a decent amount of time.",
    "Love the sleek design of my Zojirushi mug, and it fits perfectly in my car cup holder!",
    "It’s lightweight but keeps my drinks at the right temperature for hours with Hydro Flask.",
    "No more cold coffee! This Yeti thermal mug does the job.",
    "It’s easy to clean, and the thermal insulation works like a charm.",
    "The handle makes it easy to carry, and it doesn’t spill.",
    "Not great for keeping drinks cold, but excellent for hot beverages.",
    "I wish it were bigger, but it’s perfect for my morning tea.",
    "I accidentally dropped it, and it didn’t dent! Very sturdy.",
    "The rubber seal around the lid came loose after a few weeks. Disappointing.",
    "Great for both coffee and soup—keeps them warm for hours.",
    "The exterior stays cool, even when my drink is piping hot inside.",
    "I love the color options, and it’s great for on-the-go.",
    "Keeps ice water cold for hours, even in hot weather!",
    "It’s a little tricky to open one-handed, but overall, a great mug.",
    "The size is perfect for travel, and it keeps drinks hot all day.",
    "It doesn’t leak, even when I toss it in my bag. Highly recommend Contigo.",
    "The lid is a little tight, but the mug works well for keeping drinks warm.",
    "Very stylish and functional! I get compliments all the time.",
    "Keeps my coffee scalding hot for longer than any mug I’ve owned with Zojirushi.",
    "Great value for the price. Works just as well as more expensive brands.",
    "The mug is lightweight and easy to carry around.",
    "It fits perfectly under my single-serve coffee machine!",
    "Durable, sleek, and it does exactly what it’s supposed to.",

    # Dishwasher Detergents
    "My dishes come out sparkling clean every time with Cascade. Love this detergent!",
    "It works well on glass, but I’ve noticed spots on my silverware.",
    "Great for tough, greasy messes. Leaves no residue! Thanks, Finish.",
    "This detergent smells amazing and leaves my dishwasher fresh.",
    "It’s a little pricey, but my dishes have never looked better with Cascade Platinum.",
    "Gets rid of even the most stubborn baked-on food. Highly recommend Finish Quantum.",
    "Not the best on hard water stains, but otherwise it works great.",
    "My dishes have never been so spotless after a wash!",
    "It’s very effective, but I wish it came in a fragrance-free version.",
    "Cuts through grease like a dream. No more pre-rinsing with Cascade Complete.",
    "This detergent doesn’t leave any residue on plastic, which I love.",
    "My glasses come out clear and sparkling every single time.",
    "It doesn’t work well with my eco dishwasher. Dishes aren’t as clean.",
    "Very efficient—gets rid of food stains and smells with no problem.",
    "I noticed some streaks on my glassware, but overall it works well.",
    "Leaves my dishes spotless and my machine smelling fresh.",
    "A great, eco-friendly option that actually works!",
    "It’s a little hard on some of my delicate dishware.",
    "This is the only detergent that works on my hard water stains.",
    "No need to rewash dishes after using this—so efficient!",
    "Perfect for everyday use. My dishes are clean and shiny.",
    "Leaves a chemical smell, but it’s effective at cleaning.",
    "A bit expensive, but worth it for the spotless results.",
    "No more streaks or water spots! Best dishwasher detergent ever.",
    "My silverware and dishes look brand new after every wash.",

    # Sunscreens
    "Absorbs quickly and doesn’t leave a greasy residue. Great for daily use with Neutrogena.",
    "The scent is a little strong, but it protects well with Coppertone.",
    "Perfect for sensitive skin! No breakouts or irritation with La Roche-Posay.",
    "A bit thick to apply, but once it’s on, it stays all day.",
    "I love the lightweight formula of CeraVe, perfect for wearing under makeup.",
    "Doesn’t leave a white cast, even on darker skin tones.",
    "The spray bottle makes it super easy to apply on the go.",
    "This sunscreen saved me from burning on a beach vacation with Banana Boat!",
    "It’s waterproof, which is a must for pool days. Highly recommend Hawaiian Tropic.",
    "A bit pricey, but the protection it provides is worth every penny with Supergoop.",
    "This is my go-to sunscreen for both my face and body with Neutrogena.",
    "It’s a little greasy, but it gets the job done in strong sun.",
    "No weird scent, and it goes on smooth. Love this product!",
    "Perfect for outdoor activities—no sunburn, even after hours outside.",
    "It’s great for kids! No irritation, and it’s easy to apply with Blue Lizard.",
    "A little too heavy for my face, but works perfectly for the body.",
    "The texture is nice and light, not sticky at all.",
    "This sunscreen doesn’t clog my pores, which is a huge plus with EltaMD.",
    "I’ve tried a lot of sunscreens, and this one offers the best protection with La Roche-Posay.",
    "It leaves a slight sheen, but I love how protected my skin feels.",
    "This formula doesn’t dry out my skin like others do.",
    "It’s great under makeup—no pilling or greasiness.",
    "I wish it were more affordable, but it’s worth it for the protection.",
    "Very effective, even after swimming for hours.",
    "My skin stays soft and protected all day with Neutrogena sunscreen."

     # Powder Detergents for Laundry
    "Gets my clothes fresh and clean every time. No lingering odor with Tide.",
    "It dissolves well, even in cold water. My whites have never been brighter thanks to Ariel.",
    "A little pricey, but worth it for the excellent stain removal power of Persil.",
    "This powder leaves a residue on darker clothes. Not a fan of OMO.",
    "Great for sensitive skin! No itching or redness after using Seventh Generation.",
    "I love how eco-friendly this detergent is. It’s a big plus for me with Ecover.",
    "I don’t need fabric softener anymore—this leaves my clothes so soft!",
    "My laundry has never smelled so fresh, and it lasts for days with Gain.",
    "It’s not the best for heavy stains but works great for daily washes.",
    "Great value for the price. This box lasts forever! Thanks, Arm & Hammer.",
    "Perfect for my workout gear—gets rid of all the sweat smells.",
    "Leaves a bit of powder behind in the machine, but it cleans well.",
    "I’ve been using it for years, and Tide never disappoints.",
    "Not as effective in hard water areas, but still decent.",
    "My go-to detergent for all of my family’s laundry needs.",
    "I noticed some fading in my darker clothes after a few washes.",
    "It’s gentle on my baby’s clothes and skin with Dreft.",
    "Very effective at removing mud and grass stains from the kids’ clothes.",
    "I like the scent, but it might be too strong for some.",
    "No complaints so far! My clothes feel clean and fresh.",
    "Works just as well as liquid detergents but at a lower cost.",
    "A bit too perfumed for my taste, but it gets the job done.",
    "My clothes are noticeably softer and smell better than before.",
    "The box is hard to pour from, but the detergent works well.",
    "This is my new favorite detergent. So much better than the leading brand!",
]

In [ ]:
import re

def run_mistral(user_message, model="mistral-large-latest"):
    messages = [{"role": "user", "content": user_message}]
    chat_response = mistral_client.chat.complete(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
    )
    return chat_response.choices[0].message.content

def run_openai(user_message, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": user_message}]
    chat_response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
    )
    return chat_response.choices[0].message.content


def run_gemini(user_message):
    json_text = gemini_client.generate_content(
        user_message
    ).text
    return re.sub(r'(^```json|```$)', '', json_text).strip()


# define prompt template
prompt_template = """
Extract information from the following reviews:
{content}

Return a single line json format with the following JSON schema:

{{
        "product category": {{
            "type": "string",
            "enum": ["Powder Detergents for Laundry", "Thermal Mugs", "Dishwasher Detergents", "Sunscreens", "Nappies", "Others"]
        }},
        "brand": {{
            "type": "string" or N/A
        }},
        "other keywords": {{
            "type": "array",
            "items": {{
                "type": "string"
            }}
        }},

}}

"""

In [ ]:
import json


def compare_json_objects(obj1, obj2,keys_to_compare={'other keywords', 'brand', 'product category'}):
    if obj1 is None or obj2 is None:
      return 0.0
    total_fields = 0
    identical_fields = 0
    common_keys = set(obj1.keys()) & set(obj2.keys() & keys_to_compare)
    for key in common_keys:
        identical_fields += obj1[key] == obj2[key]
    #percentage_identical = (identical_fields / max(len(obj1.keys()), 1)) * 100
    percentage_identical = (identical_fields / max(len(keys_to_compare), 1)) * 100

    return percentage_identical

In [ ]:
import time
accuracy_rates = dict()
for model in [run_gemini, run_mistral, run_openai]:
    accuracy_rates[model.__name__] = []

# for each test case
for name in prompts:
    time.sleep(20)

    for model in [run_gemini, run_mistral, run_openai]:
      # define user message
      #user_message = #prompt_template.format(medical_notes=prompts[name]["medical_notes"])
      text=prompts[name]["review_content"]

      user_message = prompt_template.format(content=text)
      #print(user_message)
      #user_message = prompt_templateR.format(notes=promptsR[name]["review_content"])

      # run LLM
      try:
        response_raw = model(user_message)
        print(model.__name__, response_raw)
        response = json.loads(response_raw)
      except:
        response = None
      print(response)
      # calculate accuracy rate for this test case
      accuracy_rates[model.__name__].append(
          compare_json_objects(response, prompts[name]["golden_answer"],keys_to_compare={'product category','brand'})
      )


run_gemini {"product category": "Powder Detergents for Laundry", "brand": "Tide", "other keywords": ["fresh", "clean", "odor"]}
{'product category': 'Powder Detergents for Laundry', 'brand': 'Tide', 'other keywords': ['fresh', 'clean', 'odor']}
run_mistral {"product category": "Powder Detergents for Laundry", "brand": "Tide", "other keywords": ["fresh", "clean", "odor"]}
{'product category': 'Powder Detergents for Laundry', 'brand': 'Tide', 'other keywords': ['fresh', 'clean', 'odor']}
None
run_gemini {"product category": "Thermal Mugs", "brand": "Contigo", "other keywords": ["coffee", "long workdays", "hot"]}
{'product category': 'Thermal Mugs', 'brand': 'Contigo', 'other keywords': ['coffee', 'long workdays', 'hot']}
run_mistral {"product category": "Thermal Mugs", "brand": "Contigo", "other keywords": ["coffee", "hot", "hours", "long workdays"]}
{'product category': 'Thermal Mugs', 'brand': 'Contigo', 'other keywords': ['coffee', 'hot', 'hours', 'long workdays']}
None


In [ ]:
accuracy_rates

{'run_gemini': [100.0, 100.0], 'run_mistral': [100.0, 100.0], 'run_openai': []}

In [ ]:
# prompt: create big json dictionary with subsequent reviews and LLM answer
import time
results = dict()
for model in [run_gemini, run_mistral]:
  results[model.__name__] = dict()

for review in reviews:
  for model in [run_gemini, run_mistral]:
    #print(review)
    user_message = prompt_template.format(content=review)
    try:
      response = json.loads(model(user_message))
      results[model.__name__][review] = response
    except json.JSONDecodeError as e:
      print(model.__name__, response, e)
    #print(response)
  time.sleep(2)

import json
with open('results.json', 'w') as f:
  json.dump(results, f, indent=4)


In [ ]:
results["run_gemini"]

{'Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.': {'product category': 'Thermal Mugs',
  'brand': 'Contigo',
  'other keywords': ['coffee', 'hot', 'long workdays']},
 'The lid isn’t leak-proof, but it keeps drinks warm for a decent amount of time.': {'product category': 'Thermal Mugs',
  'brand': 'N/A',
  'other keywords': ['leak-proof', 'warm']},
 'Love the sleek design of my Zojirushi mug, and it fits perfectly in my car cup holder!': {'product category': 'Thermal Mugs',
  'brand': 'Zojirushi',
  'other keywords': ['sleek design', 'car cup holder']},
 'It’s lightweight but keeps my drinks at the right temperature for hours with Hydro Flask.': {'product category': 'Thermal Mugs',
  'brand': 'Hydro Flask',
  'other keywords': ['lightweight', 'temperature']},
 'No more cold coffee! This Yeti thermal mug does the job.': {'product category': 'Thermal Mugs',
  'brand': 'Yeti',
  'other keywords': ['cold coffee']},
 'It’s easy to clean, and the thermal i

In [ ]:
product_offers = [
    # Thermal Mugs
    "Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!",
    "Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders",
    "Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours",
    "Yeti Thermal Mug – No More Cold Coffee!",
    "Contigo All-Day Heat Retention Mug – Ideal for Travel",
    "Contigo Leak-Proof Mug – Toss in Your Bag with Confidence",
    "Zojirushi Scalding Hot Coffee Mug – Best Insulation Yet",

    # Dishwasher Detergents
    "Cascade Sparkling Clean Detergent – Your Dishes Will Shine",
    "Finish Detergent – Tough on Grease, No Residue Left",
    "Cascade Platinum Detergent – Pricey, But Worth It for Results",
    "Finish Quantum Detergent – Stubborn Food Stains Gone",
    "Cascade Complete Detergent – No Pre-Rinsing Needed for Grease",

    # Sunscreens
    "Neutrogena Daily Sunscreen – Quick Absorption, No Grease",
    "Coppertone Suncream – Strong Scent, Strong Protection",
    "La Roche-Posay Sensitive Skin Sunscreen – No Breakouts",
    "CeraVe Lightweight Sunscreen – Perfect Under Makeup",
    "Banana Boat Beach-Saver Sunscreen – No Burns, Just Fun",
    "Hawaiian Tropic Waterproof Sunscreen – Pool Day Essential",
    "Supergoop Premium Sunscreen – Worth Every Penny",
    "Neutrogena Face & Body Sunscreen – All-Purpose Protection",
    "EltaMD Pore-Friendly Sunscreen – Protection Without Clogging",
    "La Roche-Posay Suncream – The Best in Sun Protection",
    "Neutrogena Sunscreen – Soft, Protected Skin All Day",

    # Powder Detergents for Laundry
    "Tide Powder Detergent – Fresh, Clean Clothes Every Time",
    "Ariel Powder Detergent – Whites Brighter, Even in Cold Water",
    "Persil Powder Detergent – Powerful Stain Removal",
    "OMO Powder Detergent – Leaves Residue on Dark Clothes",
    "Seventh Generation Powder Detergent – Great for Sensitive Skin",
    "Ecover Eco-Friendly Detergent – Perfect for the Eco-Conscious",
    "Gain Powder Detergent – Fresh-Smelling Laundry for Days",
    "Arm & Hammer Powder Detergent – Great Value, Lasts Forever",
    "Dreft Baby Powder Detergent – Gentle on Baby Clothes"
]

In [ ]:
products = {}
products["run_mistral"] = {}
products["run_gemini"] = {}
for title in product_offers:
  for model in [run_mistral, run_gemini]:
    print(model.__name__, title)
    user_message = prompt_template.format(content=title)
    response = json.loads(model(user_message))
    products[model.__name__][title] = response
    print(response)
    time.sleep(3)
    print()

import json
with open('products.json', 'w') as f:
  json.dump(products, f, indent=4)

run_mistral Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!
{'product category': 'Thermal Mugs', 'brand': 'Contigo', 'other keywords': ['Workday', 'Travel Mug', 'Keeps Coffee Hot']}

run_gemini Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!
{'product category': 'Thermal Mugs', 'brand': 'Contigo', 'other keywords': ['Workday Travel Mug', 'Keeps Coffee Hot', 'Hours']}

run_mistral Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders
{'product category': 'Thermal Mugs', 'brand': 'Zojirushi', 'other keywords': ['Sleek Travel Mug', 'Perfect Fit for Car Holders']}

run_gemini Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders
{'product category': 'Thermal Mugs', 'brand': 'Zojirushi', 'other keywords': ['Sleek', 'Travel Mug', 'Car Holders']}

run_mistral Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours
{'product category': 'Thermal Mugs', 'brand': 'Hydro Flask', 'other keywords': ['Lightweight Insulation', 'Stay Warm for Hours']}

run_gemini Hydro Flas

In [ ]:
# prompt: write code for comparing one review from results with one product from products and add in these comparison extracted keywords with lower weight additionally to categories and brand comparison

def compare_json_objects_with_keywords(obj1, obj2, keys_to_compare={'other keywords', 'brand', 'product category'}):
    total_fields = 0
    identical_fields = 0
    common_keys = set(obj1.keys()) & set(obj2.keys() & keys_to_compare)
    for key in common_keys:
        if key == 'other keywords':
            if isinstance(obj1.get(key), list) and isinstance(obj2.get(key), list):
                common_keywords = set(obj1.get(key)) & set(obj2.get(key))
                identical_fields += len(common_keywords) * 0.3  # Keywords have lower weight
            else:
              identical_fields += 0
        else:
          identical_fields += obj1[key] == obj2[key]
    #percentage_identical = (identical_fields / max(len(obj1.keys()), 1)) * 100
    percentage_identical = (identical_fields / max(len(keys_to_compare), 1)) * 100

    return percentage_identical


# Example usage (assuming 'results' and 'products' are dictionaries)
review = list(results["run_gemini"].keys())[50]  # Get the first review
review_data = results["run_gemini"][review]

product = list(products["run_gemini"].keys())[20]  # Get the first product
product_data = products["run_gemini"][product]
print(review_data)
print(product_data)
similarity_score = compare_json_objects_with_keywords(review_data, product_data,keys_to_compare={'brand', 'product category'})
print(f"Similarity between review:\n '{review}'\n and product:\n '{product}'\n is as follow:\n {similarity_score}")


{'product category': 'Sunscreens', 'brand': 'Neutrogena', 'other keywords': ['absorbs quickly', 'greasy residue', 'daily use']}
{'product category': 'Sunscreens', 'brand': 'EltaMD', 'other keywords': ['Pore-Friendly', 'Protection', 'Clogging']}
Similarity between review:
 'Absorbs quickly and doesn’t leave a greasy residue. Great for daily use with Neutrogena.'
 and product:
 'EltaMD Pore-Friendly Sunscreen – Protection Without Clogging'
 is as follow:
 50.0


In [ ]:
# prompt: add comparison when if product has different cateogry there is no similarity, but if it is the same it is about 30%, if has the same category and brand, similarity is about 75%

def compare_json_objects_with_keywords(obj1, obj2, keys_to_compare={'other keywords', 'brand', 'product category'}):
    if obj1.get('product category') != obj2.get('product category'):
        return 0  # No similarity if categories don't match

    similarity_score = 0
    common_keys = set(obj1.keys()) & set(obj2.keys() & keys_to_compare)

    if obj1.get('product category') == obj2.get('product category'):
        similarity_score += 0.3  # Base similarity for matching categories

    if obj1.get('brand') == obj2.get('brand'):
        similarity_score += 0.4  # Additional similarity for matching brands

    for key in common_keys:
        if key == 'other keywords':
            if isinstance(obj1.get(key), list) and isinstance(obj2.get(key), list):
                common_keywords = set(obj1.get(key)) & set(obj2.get(key))
                similarity_score += len(common_keywords) * 0.05  # Keywords have lower weight
            else:
                similarity_score += 0


    return min(100, round(similarity_score * 100))


In [ ]:
# Example usage (assuming 'results' and 'products' are dictionaries)
review = list(results["run_gemini"].keys())[50]  # Get the first review
review_data = results["run_gemini"][review]

product = list(products["run_gemini"].keys())[20]  # Get the first product
product_data = products['run_gemini'][product]
print(review_data)
print(product_data)


similarity_score = compare_json_objects_with_keywords(review_data, product_data,keys_to_compare={'brand', 'product category'})
print(f"Similarity between review:\n '{review}'\n and product:\n '{product}'\n is as follow:\n {similarity_score}")


{'product category': 'Sunscreens', 'brand': 'Neutrogena', 'other keywords': ['absorbs quickly', 'greasy residue', 'daily use']}
{'product category': 'Sunscreens', 'brand': 'EltaMD', 'other keywords': ['Pore-Friendly', 'Protection', 'Clogging']}
Similarity between review:
 'Absorbs quickly and doesn’t leave a greasy residue. Great for daily use with Neutrogena.'
 and product:
 'EltaMD Pore-Friendly Sunscreen – Protection Without Clogging'
 is as follow:
 30


In [ ]:
# prompt: use compare_json_objects_with_keywords to evaluate run_gemini and run_mistral  use function defined above and calculate similarity for all matching products and reviews then aggreagte them and show means

similarity_scores_gemini = []
similarity_scores_mistral = []

for review in results["run_gemini"]:
  review_data_gemini = results["run_gemini"][review]

  for product in products["run_gemini"]:
    product_data_gemini = products["run_gemini"][product]
    similarity_score_gemini = compare_json_objects_with_keywords(review_data_gemini, product_data_gemini)
    similarity_scores_gemini.append(similarity_score_gemini)

for review in results["run_mistral"]:
  review_data_mistral = results["run_mistral"][review]

  for product in products["run_mistral"]:
    product_data_mistral = products["run_mistral"][product]
    similarity_score_mistral = compare_json_objects_with_keywords(review_data_mistral, product_data_mistral)
    similarity_scores_mistral.append(similarity_score_mistral)

print("Mean similarity score for Gemini:", sum(similarity_scores_gemini) / len(similarity_scores_gemini) if similarity_scores_gemini else 0)
print("Mean similarity score for Mistral:", sum(similarity_scores_mistral) / len(similarity_scores_mistral) if similarity_scores_mistral else 0)

Mean similarity score for Gemini: 6.1916035353535355
Mean similarity score for Mistral: 6.32260101010101


In [ ]:
# prompt: calculate similarity for all review and product pairs and for each review show similarities to products put results in a table where reviews are rows, products are columns and cells contain similarity scores make a separate table for gemini and mistral

import pandas as pd

# Assuming 'results' and 'products' are your dictionaries containing review and product data
# and 'compare_json_objects_with_keywords' is your function to calculate similarity.

def create_similarity_table(results, products, model_name):
  """Creates a similarity table for a given model."""

  reviews = list(results[model_name].keys())
  products_list = list(products[model_name].keys())

  similarity_table = pd.DataFrame(index=reviews, columns=products_list)

  for review in reviews:
    review_data = results[model_name][review]
    for product in products_list:
      product_data = products[model_name][product]
      similarity_score = compare_json_objects_with_keywords(review_data, product_data)
      similarity_table.loc[review, product] = similarity_score

  return similarity_table


# Create similarity tables for Gemini and Mistral
similarity_table_gemini = create_similarity_table(results, products, "run_gemini")
similarity_table_mistral = create_similarity_table(results, products, "run_mistral")

# Print or use the tables as needed
print("Gemini Similarity Table:")
print(similarity_table_gemini)

print("\nMistral Similarity Table:")
similarity_table_mistral

Gemini Similarity Table:
                                                   Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!  \
Keeps my coffee hot for hours—just what I need ...                                                 70         
The lid isn’t leak-proof, but it keeps drinks w...                                                 30         
Love the sleek design of my Zojirushi mug, and ...                                                 30         
It’s lightweight but keeps my drinks at the rig...                                                 30         
No more cold coffee! This Yeti thermal mug does...                                                 30         
...                                                                                               ...         
Works just as well as liquid detergents but at ...                                                  0         
A bit too perfumed for my taste, but it gets th...                                     

,Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!,Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders,Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours,Yeti Thermal Mug – No More Cold Coffee!,Contigo All-Day Heat Retention Mug – Ideal for Travel,Contigo Leak-Proof Mug – Toss in Your Bag with Confidence,Zojirushi Scalding Hot Coffee Mug – Best Insulation Yet,Cascade Sparkling Clean Detergent – Your Dishes Will Shine,"Finish Detergent – Tough on Grease, No Residue Left","Cascade Platinum Detergent – Pricey, But Worth It for Results",...,"Neutrogena Sunscreen – Soft, Protected Skin All Day","Tide Powder Detergent – Fresh, Clean Clothes Every Time","Ariel Powder Detergent – Whites Brighter, Even in Cold Water",Persil Powder Detergent – Powerful Stain Removal,OMO Powder Detergent – Leaves Residue on Dark Clothes,Seventh Generation Powder Detergent – Great for Sensitive Skin,Ecover Eco-Friendly Detergent – Perfect for the Eco-Conscious,Gain Powder Detergent – Fresh-Smelling Laundry for Days,"Arm & Hammer Powder Detergent – Great Value, Lasts Forever",Dreft Baby Powder Detergent – Gentle on Baby Clothes
"Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.",70,30,30,30,70,70,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The lid isn’t leak-proof, but it keeps drinks warm for a decent amount of time.",30,30,30,30,30,30,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Love the sleek design of my Zojirushi mug, and it fits perfectly in my car cup holder!",30,70,30,30,30,30,70,0,0,0,...,0,0,0,0,0,0,0,0,0,0
It’s lightweight but keeps my drinks at the right temperature for hours with Hydro Flask.,30,30,70,30,30,30,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
No more cold coffee! This Yeti thermal mug does the job.,30,30,30,70,30,30,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Works just as well as liquid detergents but at a lower cost.,0,0,0,0,0,0,0,0,0,0,...,0,30,30,30,30,30,0,30,30,30
"A bit too perfumed for my taste, but it gets the job done.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
My clothes are noticeably softer and smell better than before.,0,0,0,0,0,0,0,0,0,0,...,0,30,30,30,30,30,0,30,30,30
"The box is hard to pour from, but the detergent works well.",0,0,0,0,0,0,0,0,0,0,...,0,30,30,30,30,30,0,30,30,30


In [ ]:
# prompt: for each review return the best matching product from gemini and mistral for each review

best_matching_products = {}

for review in results["run_gemini"]:
  review_data_gemini = results["run_gemini"][review]
  best_similarity_gemini = 0
  best_product_gemini = None

  for product in products["run_gemini"]:
    product_data_gemini = products["run_gemini"][product]
    similarity_score_gemini = compare_json_objects_with_keywords(
        review_data_gemini, product_data_gemini
    )
    if similarity_score_gemini > best_similarity_gemini:
      best_similarity_gemini = similarity_score_gemini
      best_product_gemini = product

  review_data_mistral = results["run_mistral"][review]
  best_similarity_mistral = 0
  best_product_mistral = None

  for product in products["run_mistral"]:
    product_data_mistral = products["run_mistral"][product]
    similarity_score_mistral = compare_json_objects_with_keywords(
        review_data_mistral, product_data_mistral
    )
    if similarity_score_mistral > best_similarity_mistral:
      best_similarity_mistral = similarity_score_mistral
      best_product_mistral = product

  best_matching_products[review] = {
      "gemini": best_product_gemini,
      "mistral": best_product_mistral,
  }

best_matching_products

{'Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.': {'gemini': 'Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!',
  'mistral': 'Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!'},
 'The lid isn’t leak-proof, but it keeps drinks warm for a decent amount of time.': {'gemini': 'Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!',
  'mistral': 'Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!'},
 'Love the sleek design of my Zojirushi mug, and it fits perfectly in my car cup holder!': {'gemini': 'Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders',
  'mistral': 'Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders'},
 'It’s lightweight but keeps my drinks at the right temperature for hours with Hydro Flask.': {'gemini': 'Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours',
  'mistral': 'Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours'},
 'No more cold coffee! This Yeti thermal mug does the jo

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [ ]:
# prompt: comapre review and product keyword by bertscore

from bert_score import score

def compare_keywords_bert_score(review_keywords, product_keywords):
  """Compares keywords using BERTScore."""

  if not review_keywords or not product_keywords:
    return 0
  print(review_keywords)
  print(product_keywords)

  # Calculate BERTScore
  P, R, F1 = score(
      [keyword for keyword in review_keywords],
      [keyword for keyword in product_keywords],
      lang="en",
      model_type="bert-base-uncased",
      verbose=False
  )

  return F1.mean().item()


# Example usage
review_keywords = results["run_gemini"][review].get("other keywords", [])
product_keywords = products["run_gemini"][product].get("other keywords", [])

similarity_score_bert = compare_keywords_bert_score(review_keywords, product_keywords)

print(
    f"Similarity (BERTScore) between review:\n '{review}'\n and product:\n '{product}'\n is as follow:\n {similarity_score_bert}"
)


['coffee', 'hot', 'long workdays']
['Workday Travel Mug', 'Keeps Coffee Hot', 'Hours']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Similarity (BERTScore) between review:
 'Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.'
 and product:
 'Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!'
 is as follow:
 0.5237007141113281


In [ ]:
review =  'Absorbs quickly and doesn’t leave a greasy residue. Great for daily use with Neutrogena.'
product = 'EltaMD Pore-Friendly Sunscreen – Protection Without Clogging'
review_data = {'product category': 'Sunscreens', 'brand': 'Neutrogena', 'other keywords': ['spf50', 'daily use']}
product_data = {'product category': 'Sunscreens', 'brand': 'Neutrogena', 'other keywords': ['spf50','Pore-Friendly', 'Protection', 'daily use']}

similarity_score = compare_json_objects_with_keywords(review_data, product_data,keys_to_compare={'other keywords','brand', 'product category'})
print(f"Similarity between review:\n '{review}'\n and product:\n '{product}'\n is as follow:\n {similarity_score}")

similarity_score_bert = compare_keywords_bert_score(review_keywords, product_keywords)

print(
    f"Similarity (BERTScore) between review:\n '{review}'\n and product:\n '{product}'\n is as follow:\n {similarity_score_bert}"
)


Similarity between review:
 'Absorbs quickly and doesn’t leave a greasy residue. Great for daily use with Neutrogena.'
 and product:
 'EltaMD Pore-Friendly Sunscreen – Protection Without Clogging'
 is as follow:
 80
['coffee', 'hot', 'long workdays']
['Workday Travel Mug', 'Keeps Coffee Hot', 'Hours']
Similarity (BERTScore) between review:
 'Absorbs quickly and doesn’t leave a greasy residue. Great for daily use with Neutrogena.'
 and product:
 'EltaMD Pore-Friendly Sunscreen – Protection Without Clogging'
 is as follow:
 0.5237007141113281


In [ ]:
# prompt: calculate a bert score table where each row is a review each column a product and cell are bert_scores make a separate table for gemini and mistral

bert_scores_gemini_table_data = {}

for review in results["run_gemini"]:
  review_data_gemini = results["run_gemini"][review]
  bert_scores_gemini_table_data[review] = {}

  for product in products["run_gemini"]:
    product_data_gemini = products["run_gemini"][product]

    review_keywords = review_data_gemini.get("other keywords", [])
    product_keywords = product_data_gemini.get("other keywords", [])
    try:
      bert_score = compare_keywords_bert_score(review_keywords, product_keywords)
    except:
      bert_score = -1
    bert_scores_gemini_table_data[review][product] = bert_score

# Convert the data to a Pandas DataFrame for better visualization and manipulation
bert_scores_gemini_df = pd.DataFrame(bert_scores_gemini_table_data)

# Transpose the DataFrame to have reviews as rows and products as columns
bert_scores_gemini_df = bert_scores_gemini_df.transpose()

# Display the similarity table
bert_scores_gemini_df


bert_scores_mistral_table_data = {}

for review in results["run_mistral"]:
  review_data_mistral = results["run_mistral"][review]
  bert_scores_mistral_table_data[review] = {}

  for product in products["run_mistral"]:
    product_data_mistral = products["run_mistral"][product]

    review_keywords = review_data_mistral.get("other keywords", [])
    product_keywords = product_data_mistral.get("other keywords", [])
    try:
      bert_score = compare_keywords_bert_score(review_keywords, product_keywords)
    except:
      bert_score = -1
    bert_scores_mistral_table_data[review][product] = bert_score

# Convert the data to a Pandas DataFrame for better visualization and manipulation
bert_scores_mistral_df = pd.DataFrame(bert_scores_mistral_table_data)

# Transpose the DataFrame to have reviews as rows and products as columns
bert_scores_mistral_df = bert_scores_mistral_df.transpose()

# Display the similarity table
bert_scores_mistral_df

['coffee', 'hot', 'long workdays']
['Workday Travel Mug', 'Keeps Coffee Hot', 'Hours']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Streaming output truncated to the last 5000 lines.
['value', 'price', 'expensive']
['Fresh-Smelling Laundry', 'Days']
['value', 'price', 'expensive']
['Great Value', 'Lasts Forever']
['value', 'price', 'expensive']
['Baby Clothes', 'Gentle']
['lightweight', 'easy to carry']
['Workday Travel Mug', 'Keeps Coffee Hot', 'Hours']
['lightweight', 'easy to carry']
['Sleek', 'Travel Mug', 'Car Holders']
['lightweight', 'easy to carry']
['Lightweight Insulation', 'Stay Warm', 'Hours']
['lightweight', 'easy to carry']
['Cold Coffee']
['lightweight', 'easy to carry']
['All-Day Heat Retention', 'Travel']
['lightweight', 'easy to carry']
['Leak-Proof', 'Toss in Your Bag']
['lightweight', 'easy to carry']
['insulation', 'coffee', 'hot']
['lightweight', 'easy to carry']
['Sparkling Clean', 'Dishes', 'Shine']
['lightweight', 'easy to carry']
['Tough on Grease', 'No Residue Left']
['lightweight', 'easy to carry']
['Pricey', 'Worth It']
['lightweight', 'easy to carry']
['Quantum', 'Stubborn Food Stains'

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Streaming output truncated to the last 5000 lines.
['stylish', 'functional', 'compliments']
['Eco-Friendly', 'Eco-Conscious']
['stylish', 'functional', 'compliments']
['Fresh-Smelling Laundry for Days']
['stylish', 'functional', 'compliments']
['Great Value', 'Lasts Forever']
['stylish', 'functional', 'compliments']
['Baby Powder Detergent', 'Gentle on Baby Clothes']
['scalding hot', 'longer']
['Workday', 'Travel Mug', 'Keeps Coffee Hot']
['scalding hot', 'longer']
['Sleek Travel Mug', 'Perfect Fit for Car Holders']
['scalding hot', 'longer']
['Lightweight Insulation', 'Stay Warm for Hours']
['scalding hot', 'longer']
['No More Cold Coffee']
['scalding hot', 'longer']
['All-Day Heat Retention Mug', 'Ideal for Travel']
['scalding hot', 'longer']
['Leak-Proof', 'Bag', 'Confidence']
['scalding hot', 'longer']
['Scalding Hot', 'Coffee Mug', 'Best Insulation']
['scalding hot', 'longer']
['Sparkling Clean', 'Dishes', 'Shine']
['scalding hot', 'longer']
['Tough on Grease', 'No Residue Left']


,Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!,Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders,Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours,Yeti Thermal Mug – No More Cold Coffee!,Contigo All-Day Heat Retention Mug – Ideal for Travel,Contigo Leak-Proof Mug – Toss in Your Bag with Confidence,Zojirushi Scalding Hot Coffee Mug – Best Insulation Yet,Cascade Sparkling Clean Detergent – Your Dishes Will Shine,"Finish Detergent – Tough on Grease, No Residue Left","Cascade Platinum Detergent – Pricey, But Worth It for Results",...,"Neutrogena Sunscreen – Soft, Protected Skin All Day","Tide Powder Detergent – Fresh, Clean Clothes Every Time","Ariel Powder Detergent – Whites Brighter, Even in Cold Water",Persil Powder Detergent – Powerful Stain Removal,OMO Powder Detergent – Leaves Residue on Dark Clothes,Seventh Generation Powder Detergent – Great for Sensitive Skin,Ecover Eco-Friendly Detergent – Perfect for the Eco-Conscious,Gain Powder Detergent – Fresh-Smelling Laundry for Days,"Arm & Hammer Powder Detergent – Great Value, Lasts Forever",Dreft Baby Powder Detergent – Gentle on Baby Clothes
"Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.",-1.00000,-1.000000,-1.000000,-1.00000,-1.000000,-1.00000,-1.00000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
"The lid isn’t leak-proof, but it keeps drinks warm for a decent amount of time.",0.44861,-1.000000,-1.000000,-1.00000,-1.000000,0.66897,0.48088,0.475226,-1.000000,0.390062,...,0.508333,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
"Love the sleek design of my Zojirushi mug, and it fits perfectly in my car cup holder!",-1.00000,0.675142,0.548372,-1.00000,0.445723,-1.00000,-1.00000,-1.000000,0.451219,-1.000000,...,-1.000000,0.519808,0.461170,0.556135,0.503378,-1.000000,0.443410,-1.000000,0.528807,0.471186
It’s lightweight but keeps my drinks at the right temperature for hours with Hydro Flask.,-1.00000,0.452180,0.601692,-1.00000,0.409924,-1.00000,-1.00000,-1.000000,0.390011,-1.000000,...,-1.000000,0.513994,0.458049,0.483291,0.424139,-1.000000,0.329812,-1.000000,0.440326,0.385309
No more cold coffee! This Yeti thermal mug does the job.,-1.00000,-1.000000,-1.000000,0.71308,-1.000000,-1.00000,-1.00000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.463487,-1.000000,0.439517,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Works just as well as liquid detergents but at a lower cost.,-1.00000,0.460847,0.457253,-1.00000,0.464086,-1.00000,-1.00000,-1.000000,0.431725,-1.000000,...,-1.000000,0.499114,0.417918,0.447636,0.539483,-1.000000,0.434928,-1.000000,0.438166,0.563744
"A bit too perfumed for my taste, but it gets the job done.",-1.00000,0.470796,0.429782,-1.00000,0.382091,-1.00000,-1.00000,-1.000000,0.417909,-1.000000,...,-1.000000,0.541908,0.455461,0.554491,0.532201,-1.000000,0.413435,-1.000000,0.480880,0.421294
My clothes are noticeably softer and smell better than before.,-1.00000,0.447007,0.453936,-1.00000,0.358141,-1.00000,-1.00000,-1.000000,0.416221,-1.000000,...,-1.000000,0.618224,0.456635,0.611302,0.512612,-1.000000,0.341886,-1.000000,0.473381,0.353493
"The box is hard to pour from, but the detergent works well.",-1.00000,0.446142,0.413060,-1.00000,0.385982,-1.00000,-1.00000,-1.000000,0.504254,-1.000000,...,-1.000000,0.489615,0.392700,0.469331,0.458776,-1.000000,0.345152,-1.000000,0.510795,0.359975


In [ ]:
# prompt: for each review print the best matching product by gemini and mistral

best_matching_products_with_bert_score = {}

for review in results["run_gemini"]:
    review_data_gemini = results["run_gemini"][review]
    best_similarity_gemini = 0
    best_product_gemini = None
    best_bert_score_gemini = 0


    for product in products["run_gemini"]:
        product_data_gemini = products["run_gemini"][product]
        similarity_score_gemini = compare_json_objects_with_keywords(
            review_data_gemini, product_data_gemini
        )
        review_keywords = review_data_gemini.get("other keywords", [])
        product_keywords = product_data_gemini.get("other keywords", [])
        try:
          bert_score = compare_keywords_bert_score(review_keywords, product_keywords)
        except:
          bert_score = 0

        if similarity_score_gemini > best_similarity_gemini:
            best_similarity_gemini = similarity_score_gemini
            best_product_gemini = product
            best_bert_score_gemini = bert_score

    review_data_mistral = results["run_mistral"][review]
    best_similarity_mistral = 0
    best_product_mistral = None
    best_bert_score_mistral = 0

    for product in products["run_mistral"]:
        product_data_mistral = products["run_mistral"][product]
        similarity_score_mistral = compare_json_objects_with_keywords(
            review_data_mistral, product_data_mistral
        )
        review_keywords = review_data_mistral.get("other keywords", [])
        product_keywords = product_data_mistral.get("other keywords", [])
        try:
          bert_score = compare_keywords_bert_score(review_keywords, product_keywords)
        except:
          bert_score = 0

        if similarity_score_mistral > best_similarity_mistral:
            best_similarity_mistral = similarity_score_mistral
            best_product_mistral = product
            best_bert_score_mistral = bert_score


    best_matching_products_with_bert_score[review] = {
        "gemini": {
            "product": best_product_gemini,
            "similarity_score": best_similarity_gemini,
            "bert_score": best_bert_score_gemini
        },
        "mistral": {
            "product": best_product_mistral,
            "similarity_score": best_similarity_mistral,
            "bert_score": best_bert_score_mistral
        }
    }

for review, matching_products in best_matching_products_with_bert_score.items():
  print(f"Review: {review}")
  print(f"  Gemini best matching product: {matching_products['gemini']['product']} (Similarity: {matching_products['gemini']['similarity_score']}, BERT Score: {matching_products['gemini']['bert_score']})")
  print(f"  Mistral best matching product: {matching_products['mistral']['product']} (Similarity: {matching_products['mistral']['similarity_score']}, BERT Score: {matching_products['mistral']['bert_score']})")
  print("---")

['coffee', 'hot', 'long workdays']
['Workday Travel Mug', 'Keeps Coffee Hot', 'Hours']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Streaming output truncated to the last 5000 lines.
['weird scent', 'goes on smooth', 'love this product']
['Lightweight Insulation', 'Stay Warm for Hours']
['weird scent', 'goes on smooth', 'love this product']
['No More Cold Coffee']
['weird scent', 'goes on smooth', 'love this product']
['All-Day Heat Retention Mug', 'Ideal for Travel']
['weird scent', 'goes on smooth', 'love this product']
['Leak-Proof', 'Bag', 'Confidence']
['weird scent', 'goes on smooth', 'love this product']
['Scalding Hot', 'Coffee Mug', 'Best Insulation']
['weird scent', 'goes on smooth', 'love this product']
['Sparkling Clean', 'Dishes', 'Shine']
['weird scent', 'goes on smooth', 'love this product']
['Tough on Grease', 'No Residue Left']
['weird scent', 'goes on smooth', 'love this product']
['Platinum', 'Pricey', 'Results']
['weird scent', 'goes on smooth', 'love this product']
['Stubborn Food Stains']
['weird scent', 'goes on smooth', 'love this product']
['Complete', 'No Pre-Rinsing Needed', 'Grease']
['w